**1 处理前的准备**

1.1 导入必要的库

In [121]:
import os
import json
import pickle
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from itertools import combinations

1.2 确保输出目录存在

In [122]:
input_path = Path('../data/interim/cleaned_data.pkl')
base_output_dir = Path("../data/advanced_analysis_results")

# 确保输出目录存在
try:
    os.makedirs(base_output_dir, exist_ok=True)
    print(f"创建或确认 {base_output_dir} 目录成功")
except Exception as e:
    print(f"创建 {base_output_dir} 目录失败: {e}")
    exit()

# 加载 cleaned_data
input_path = Path('../data/interim/cleaned_data.pkl')
if input_path.exists():
    with open(input_path, 'rb') as f:
        cleaned_data = pickle.load(f)
    print("✅ 成功加载 cleaned_data")
else:
    raise FileNotFoundError(f"未找到 cleaned_data.pkl，请先运行 02-cleaning.ipynb 并保存数据")

创建或确认 ..\data\advanced_analysis_results 目录成功
✅ 成功加载 cleaned_data


1.3 初始化分析结果

In [123]:
analysis_results = {
    "rating_trends": {},
    "rating_feature_relations": {},
    "interest_cycles": {},
    "consumption_speed": {},
    "genre_cooccurrence": {},
    "rating_prediction": {}
}

**2 分析处理**

2.1 评分趋势

In [124]:
def analyze_rating_trends(df, category, sheet_name):
    trends = {"douban_rating": {}, "my_rating": {}}
    if '创建时间' in df.columns and '豆瓣评分' in df.columns:
        df['创建时间'] = pd.to_datetime(df['创建时间'], errors='coerce')
        df['季度'] = df['创建时间'].dt.to_period('Q')
        # 豆瓣评分
        douban_trend = df.groupby('季度')['豆瓣评分'].agg(['mean', 'count']).dropna()
        trends["douban_rating"] = {str(k): {"mean": float(v['mean']), "count": int(v['count'])} for k, v in douban_trend.iterrows()}
        # 我的评分（10分制）
        if '我的评分' in df.columns:
            df['我的评分_10'] = pd.to_numeric(df['我的评分'], errors='coerce') * 2
            my_trend = df.groupby('季度')['我的评分_10'].agg(['mean', 'count']).dropna()
            trends["my_rating"] = {str(k): {"mean": float(v['mean']), "count": int(v['count'])} for k, v in my_trend.iterrows()}
    return trends

In [125]:
# 分析评分趋势
for category, sheet in [("movies", "看过"), ("books", "读过"), ("games", "玩过")]:
    if sheet in cleaned_data:
        analysis_results["rating_trends"][category] = analyze_rating_trends(cleaned_data[sheet], category, sheet)
    else:
        analysis_results["rating_trends"][category] = {"note": f"未找到 {sheet} 数据"}

In [126]:
# 展示评分趋势
for category in ["movies", "books", "games"]:
    trends = analysis_results["rating_trends"][category]
    print(f"\n📈 {category.capitalize()} 评分趋势：")
    if "note" in trends:
        print(f"  {trends['note']}")
    else:
        print("  豆瓣评分（前3个季度）：")
        for period, stats in list(trends["douban_rating"].items())[:3]:
            print(f"    - {period}: 平均 {stats['mean']:.2f}, 数量 {stats['count']}")
        print("  我的评分（前3个季度）：")
        for period, stats in list(trends["my_rating"].items())[:3]:
            print(f"    - {period}: 平均 {stats['mean']:.2f}, 数量 {stats['count']}")


📈 Movies 评分趋势：
  豆瓣评分（前3个季度）：
    - 2021Q1: 平均 8.45, 数量 2
    - 2021Q4: 平均 8.21, 数量 21
    - 2022Q1: 平均 8.52, 数量 14
  我的评分（前3个季度）：
    - 2021Q1: 平均 10.00, 数量 2
    - 2021Q4: 平均 9.33, 数量 21
    - 2022Q1: 平均 9.14, 数量 14

📈 Books 评分趋势：
  豆瓣评分（前3个季度）：
    - 2022Q3: 平均 8.38, 数量 5
    - 2023Q2: 平均 8.36, 数量 5
    - 2023Q3: 平均 9.10, 数量 1
  我的评分（前3个季度）：
    - 2022Q3: 平均 9.60, 数量 5
    - 2023Q2: 平均 8.00, 数量 5
    - 2023Q3: 平均 8.00, 数量 1

📈 Games 评分趋势：
  豆瓣评分（前3个季度）：
    - 2025Q2: 平均 8.70, 数量 15
  我的评分（前3个季度）：
    - 2025Q2: 平均 9.47, 数量 15


2.2 评分与特征关系

In [127]:
def analyze_rating_features(df, category, sheet_name):
    features = {"genre": {}, "year": {}, "country": {}}
    if df.empty:
        return features
    
    # 类型
    if '类型' in df.columns:
        df_exploded = df.explode('类型')
        genre_stats = df_exploded.groupby('类型').agg({
            '豆瓣评分': [('mean', 'mean'), ('count', 'count')],
            '我的评分': [('my_mean', lambda x: (pd.to_numeric(x, errors='coerce') * 2).mean())]
        }).dropna()
        for genre, stats in genre_stats.iterrows():
            features["genre"][genre] = {
                "douban_mean": float(stats[('豆瓣评分', 'mean')]) if not pd.isna(stats[('豆瓣评分', 'mean')]) else None,
                "my_mean": float(stats[('我的评分', 'my_mean')]) if not pd.isna(stats[('我的评分', 'my_mean')]) else None,
                "count": int(stats[('豆瓣评分', 'count')])
            }
    
    # 年份（图书用出版年份）
    year_col = '出版年份' if category == "books" else '年份' if category == "movies" else '发售时间'
    if year_col in df.columns:
        if year_col == '发售时间':
            df['发售年份'] = pd.to_datetime(df['发售时间'], errors='coerce').dt.year
            year_col = '发售年份'
        year_stats = df.groupby(year_col).agg({
            '豆瓣评分': [('mean', 'mean'), ('count', 'count')],
            '我的评分': [('my_mean', lambda x: (pd.to_numeric(x, errors='coerce') * 2).mean())]
        }).dropna()
        for year, stats in year_stats.iterrows():
            features["year"][str(year)] = {
                "douban_mean": float(stats[('豆瓣评分', 'mean')]) if not pd.isna(stats[('豆瓣评分', 'mean')]) else None,
                "my_mean": float(stats[('我的评分', 'my_mean')]) if not pd.isna(stats[('我的评分', 'my_mean')]) else None,
                "count": int(stats[('豆瓣评分', 'count')])
            }
    
    # 国家/地区（仅影视）
    if category == "movies" and '国家/地区' in df.columns:
        df_exploded = df.explode('国家/地区')
        country_stats = df_exploded.groupby('国家/地区').agg({
            '豆瓣评分': [('mean', 'mean'), ('count', 'count')],
            '我的评分': [('my_mean', lambda x: (pd.to_numeric(x, errors='coerce') * 2).mean())]
        }).dropna()
        for country, stats in country_stats.iterrows():
            features["country"][country] = {
                "douban_mean": float(stats[('豆瓣评分', 'mean')]) if not pd.isna(stats[('豆瓣评分', 'mean')]) else None,
                "my_mean": float(stats[('我的评分', 'my_mean')]) if not pd.isna(stats[('我的评分', 'my_mean')]) else None,
                "count": int(stats[('豆瓣评分', 'count')])
            }
    
    return features

In [128]:
# 分析评分与特征关系
for category, sheet in [("movies", "看过"), ("books", "读过"), ("games", "玩过")]:
    if sheet in cleaned_data:
        analysis_results["rating_feature_relations"][category] = analyze_rating_features(cleaned_data[sheet], category, sheet)
    else:
        analysis_results["rating_feature_relations"][category] = {"note": f"未找到 {sheet} 数据"}

In [129]:
# 展示评分与特征关系
for category in ["movies", "books", "games"]:
    relations = analysis_results["rating_feature_relations"][category]
    print(f"\n🔗 {category.capitalize()} 评分与特征关系：")
    if "note" in relations:
        print(f"  {relations['note']}")
    else:
        print("  类型（前3个）：")
        for genre, stats in list(relations["genre"].items())[:3]:
            douban_str = f"{stats['douban_mean']:.2f}" if stats['douban_mean'] is not None else "N/A"
            my_str = f"{stats['my_mean']:.2f}" if stats['my_mean'] is not None else "N/A"
            print(f"    - {genre}: 豆瓣 {douban_str}, 我的评分 {my_str}, 数量 {stats['count']}")
        print("  年份（前3个）：")
        for year, stats in list(relations["year"].items())[:3]:
            douban_str = f"{stats['douban_mean']:.2f}" if stats['douban_mean'] is not None else "N/A"
            my_str = f"{stats['my_mean']:.2f}" if stats['my_mean'] is not None else "N/A"
            print(f"    - {year}: 豆瓣 {douban_str}, 我的评分 {my_str}, 数量 {stats['count']}")
        if relations.get("country"):
            print("  国家/地区（前3个）：")
            for country, stats in list(relations["country"].items())[:3]:
                douban_str = f"{stats['douban_mean']:.2f}" if stats['douban_mean'] is not None else "N/A"
                my_str = f"{stats['my_mean']:.2f}" if stats['my_mean'] is not None else "N/A"
                print(f"    - {country}: 豆瓣 {douban_str}, 我的评分 {my_str}, 数量 {stats['count']}")


🔗 Movies 评分与特征关系：
  类型（前3个）：
    - 传记: 豆瓣 8.24, 我的评分 8.67, 数量 9
    - 儿童: 豆瓣 7.20, 我的评分 10.00, 数量 1
    - 冒险: 豆瓣 7.60, 我的评分 8.25, 数量 48
  年份（前3个）：
    - 1994: 豆瓣 9.60, 我的评分 10.00, 数量 2
    - 1997: 豆瓣 9.15, 我的评分 10.00, 数量 2
    - 1998: 豆瓣 9.35, 我的评分 9.00, 数量 2
  国家/地区（前3个）：
    - 中国台湾: 豆瓣 8.14, 我的评分 9.20, 数量 5
    - 中国大陆: 豆瓣 7.32, 我的评分 8.38, 数量 68
    - 中国香港: 豆瓣 7.28, 我的评分 8.42, 数量 19

🔗 Books 评分与特征关系：
  类型（前3个）：
  年份（前3个）：
    - 2003: 豆瓣 9.00, 我的评分 10.00, 数量 2
    - 2006: 豆瓣 9.10, 我的评分 10.00, 数量 1
    - 2007: 豆瓣 8.10, 我的评分 10.00, 数量 1

🔗 Games 评分与特征关系：
  类型（前3个）：
    - 冒险: 豆瓣 8.86, 我的评分 9.20, 数量 10
    - 动作: 豆瓣 8.92, 我的评分 9.50, 数量 8
    - 射击: 豆瓣 8.83, 我的评分 9.33, 数量 3
  年份（前3个）：
    - 2007: 豆瓣 6.50, 我的评分 10.00, 数量 1
    - 2009: 豆瓣 9.00, 我的评分 8.00, 数量 1
    - 2011: 豆瓣 8.60, 我的评分 10.00, 数量 1


2.3 兴趣周期

In [130]:
def analyze_interest_cycles(df, category, sheet_name):
    cycles = {}
    if df.empty or '创建时间' not in df.columns:
        return {"note": f"{sheet_name} 数据为空或缺少‘创建时间’列"}
    
    df = df.copy()  # 避免修改原始数据
    try:
        df['创建时间'] = pd.to_datetime(df['创建时间'], errors='coerce')
        if df['创建时间'].isna().all():
            return {"note": f"{sheet_name} 的‘创建时间’列无效"}
        df['季度'] = df['创建时间'].dt.to_period('Q')
        cycle_counts = df['季度'].value_counts().sort_index()
        cycles = {str(k): int(v) for k, v in cycle_counts.items() if pd.notna(k)}
        if not cycles:
            return {"note": f"{sheet_name} 无有效季度数据"}
    except Exception as e:
        return {"note": f"{sheet_name} 处理‘创建时间’失败: {str(e)}"}
    
    return cycles

In [131]:
# 分析兴趣周期（包括所有 sheet）
analysis_results["interest_cycles"] = {}  # 确保初始化
for category, sheets in [
    ("movies", ["想看", "看过", "所有影视作品"]),
    ("books", ["想读", "读过", "所有图书"]),
    ("games", ["想玩", "在玩", "玩过", "所有游戏"])
]:
    analysis_results["interest_cycles"][category] = {}
    for sheet in sheets:
        if sheet in cleaned_data and not cleaned_data[sheet].empty:
            # print(f"  分析 {sheet}（记录数：{len(cleaned_data[sheet])}）")
            result = analyze_interest_cycles(cleaned_data[sheet], category, sheet)
            analysis_results["interest_cycles"][category][sheet] = result
            # print(f"    结果：{result}")
        elif sheet.startswith("所有"):
            # print(f"  合并 {sheet}...")
            combined_df = pd.concat(
                [cleaned_data[s] for s in sheets[:-1] if s in cleaned_data and not cleaned_data[s].empty],
                ignore_index=True
            )
            if combined_df.empty:
                analysis_results["interest_cycles"][category][sheet] = {"note": f"{sheet} 合并数据为空"}
                # print(f"    结果：{sheet} 合并数据为空")
            else:
                # print(f"    合并记录数：{len(combined_df)}")
                result = analyze_interest_cycles(combined_df, category, sheet)
                analysis_results["interest_cycles"][category][sheet] = result
                # print(f"    结果：{result}")
        else:
            analysis_results["interest_cycles"][category][sheet] = {"note": f"未找到 {sheet} 数据或数据为空"}
            # print(f"    结果：未找到 {sheet} 数据或数据为空")

In [132]:
# 展示兴趣周期
print("\n📅 展示兴趣周期结果：")
for category in ["movies", "books", "games"]:
    cycles = analysis_results["interest_cycles"].get(category, {})
    print(f"\n📅 {category.capitalize()} 兴趣周期：")
    if not cycles:
        print(f"  错误：{category} 兴趣周期数据缺失")
        continue
    for sheet, periods in cycles.items():
        print(f"  {sheet}（前3个季度）：")
        if periods is None:
            print(f"    错误：{sheet} 数据为 None")
        elif isinstance(periods, dict) and "note" in periods:
            print(f"    {periods['note']}")
        elif isinstance(periods, dict):
            for period, count in list(periods.items())[:3]:
                print(f"    - {period}: {count} 条记录")
        else:
            print(f"    错误：{sheet} 数据格式无效（{type(periods)}）")


📅 展示兴趣周期结果：

📅 Movies 兴趣周期：
  想看（前3个季度）：
    - 2021Q1: 1 条记录
    - 2021Q3: 1 条记录
    - 2021Q4: 5 条记录
  看过（前3个季度）：
    - 2021Q1: 2 条记录
    - 2021Q4: 21 条记录
    - 2022Q1: 14 条记录
  所有影视作品（前3个季度）：
    - 2021Q1: 3 条记录
    - 2021Q3: 1 条记录
    - 2021Q4: 26 条记录

📅 Books 兴趣周期：
  想读（前3个季度）：
    - 2021Q4: 1 条记录
    - 2022Q2: 9 条记录
    - 2022Q3: 3 条记录
  读过（前3个季度）：
    - 2022Q3: 5 条记录
    - 2023Q2: 5 条记录
    - 2023Q3: 1 条记录
  所有图书（前3个季度）：
    - 2021Q4: 1 条记录
    - 2022Q2: 9 条记录
    - 2022Q3: 8 条记录

📅 Games 兴趣周期：
  想玩（前3个季度）：
    - 2025Q2: 3 条记录
  在玩（前3个季度）：
    - 2025Q2: 2 条记录
  玩过（前3个季度）：
    - 2025Q2: 15 条记录
  所有游戏（前3个季度）：
    - 2025Q2: 20 条记录


2.4 消费速度

In [133]:
def analyze_consumption_speed(df, sheet_name):
    speed = {"yearly": {}, "quarterly": {}}
    if df.empty or '创建时间' not in df.columns:
        return {"note": f"{sheet_name} 数据为空或缺少‘创建时间’列"}
    
    df = df.copy()  # 避免修改原始数据
    try:
        df['创建时间'] = pd.to_datetime(df['创建时间'], errors='coerce')
        if df['创建时间'].isna().all():
            return {"note": f"{sheet_name} 的‘创建时间’列无效"}
        df['年份'] = df['创建时间'].dt.year
        df['季度'] = df['创建时间'].dt.to_period('Q')
        yearly_counts = df['年份'].value_counts().sort_index()
        quarterly_counts = df['季度'].value_counts().sort_index()
        speed["yearly"] = {str(k): int(v) for k, v in yearly_counts.items() if pd.notna(k)}
        speed["quarterly"] = {str(k): int(v) for k, v in quarterly_counts.items() if pd.notna(k)}
        if not speed["yearly"] and not speed["quarterly"]:
            return {"note": f"{sheet_name} 无有效年份或季度数据"}
    except Exception as e:
        return {"note": f"{sheet_name} 处理‘创建时间’失败: {str(e)}"}
    
    return speed

In [134]:
# 分析消费速度
analysis_results["consumption_speed"] = {}  # 确保初始化
for category, sheet in [("movies", "看过"), ("books", "读过"), ("games", "玩过")]:
    analysis_results["consumption_speed"][category] = {}
    # print(f"\n处理 {category} 消费速度：")
    if sheet in cleaned_data and not cleaned_data[sheet].empty:
        # print(f"  分析 {sheet}（记录数：{len(cleaned_data[sheet])}）")
        result = analyze_consumption_speed(cleaned_data[sheet], sheet)
        analysis_results["consumption_speed"][category] = result
        # print(f"    结果：{result}")
    else:
        analysis_results["consumption_speed"][category] = {"note": f"未找到 {sheet} 数据或数据为空"}
        # print(f"    结果：未找到 {sheet} 数据或数据为空")


In [135]:
# 展示消费速度
for category in ["movies", "books", "games"]:
    speed = analysis_results["consumption_speed"].get(category, None)
    print(f"\n🚀 {category.capitalize()} 消费速度：")
    if speed is None:
        print(f"  错误：{category} 消费速度数据缺失")
    elif isinstance(speed, dict) and "note" in speed:
        print(f"  {speed['note']}")
    elif isinstance(speed, dict):
        print("  年均（前3年）：")
        for year, count in list(speed["yearly"].items())[:3]:
            print(f"    - {year}: {count} 条")
        print("  季均（前3个季度）：")
        for period, count in list(speed["quarterly"].items())[:3]:
            print(f"    - {period}: {count} 条")
    else:
        print(f"  错误：{category} 数据格式无效（{type(speed)}）")


🚀 Movies 消费速度：
  年均（前3年）：
    - 2021: 23 条
    - 2022: 92 条
    - 2023: 43 条
  季均（前3个季度）：
    - 2021Q1: 2 条
    - 2021Q4: 21 条
    - 2022Q1: 14 条

🚀 Books 消费速度：
  年均（前3年）：
    - 2022: 5 条
    - 2023: 7 条
    - 2024: 12 条
  季均（前3个季度）：
    - 2022Q3: 5 条
    - 2023Q2: 5 条
    - 2023Q3: 1 条

🚀 Games 消费速度：
  年均（前3年）：
    - 2025: 15 条
  季均（前3个季度）：
    - 2025Q2: 15 条


2.5 类型共现网络

In [136]:
def analyze_genre_cooccurrence(df, category):
    cooccurrence = {}
    if '类型' in df.columns:
        for types in df['类型'].dropna():
            if isinstance(types, list):
                for g1, g2 in combinations(sorted(types), 2):
                    pair = f"{g1}-{g2}"
                    cooccurrence[pair] = cooccurrence.get(pair, 0) + 1
    return {k: {"count": v, "genres": k.split("-")} for k, v in cooccurrence.items()}

In [137]:
# 分析类型共现网络（使用合并数据）
for category, sheets in [
    ("movies", ["想看", "看过"]),
]:
    combined_df = pd.concat([cleaned_data[s] for s in sheets if s in cleaned_data], ignore_index=True)
    analysis_results["genre_cooccurrence"][category] = analyze_genre_cooccurrence(combined_df, category)

In [138]:
# 展示类型共现网络
for category in ["movies"]:
    cooccurrence = analysis_results["genre_cooccurrence"][category]
    print(f"\n🌐 {category.capitalize()} 类型共现网络（前3对）：")
    for pair, stats in list(cooccurrence.items())[:3]:
        print(f"    - {pair}: 共现 {stats['count']} 次")


🌐 Movies 类型共现网络（前3对）：
    - 剧情-犯罪: 共现 31 次
    - 动作-惊悚: 共现 16 次
    - 动作-犯罪: 共现 14 次


**3 保存分析结果到 JSON**

In [139]:
json_path = base_output_dir / "advanced_analysis_results.json"
try:
    print(f"\n调试：准备保存高级分析结果到 JSON")
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(analysis_results, f, ensure_ascii=False, indent=2)
    print(f"保存高级分析结果到 {json_path}")
except Exception as e:
    print(f"保存高级分析结果失败: {str(e)}")
    print("调试：analysis_results 示例内容：")
    print(json.dumps({"rating_trends": analysis_results["rating_trends"]}, ensure_ascii=False, indent=2))

print("\n🔍 高级分析完成！")


调试：准备保存高级分析结果到 JSON
保存高级分析结果到 ..\data\advanced_analysis_results\advanced_analysis_results.json

🔍 高级分析完成！
